In [12]:
import numpy as np
import matplotlib.pyplot as plt
from keras.layers import Input,Dense,Flatten,MaxPool1D,Activation,LeakyReLU,LSTM,BatchNormalization,Dropout, Conv1D,Lambda
from keras.models import Model
import pandas as pd
import keras.backend as K
import h5py

In [4]:
feat_file='../project_features/base_scrnn_features.h5'
fid=h5py.File(feat_file,'r');
x_tr=fid['x_tr']
y_tr=fid['y_tr']
snr_tr = fid['snr_tr']
x_ts=fid['x_ts']
y_ts=fid['y_ts']
snr_ts = fid['snr_ts']

In [22]:
def model1():
    inputs = Input((128,2,))
    l = BatchNormalization()(inputs)
#     l = Lambda(lambda t: K.expand_dims(t, -2))(l)
    l = Conv1D(filters=128,kernel_size=5,activation='relu')(l)
    l = MaxPool1D(3)(l)
    l = Conv1D(filters=128,kernel_size=5,activation='relu')(l)
#     l = Lambda(lambda t: K.squeeze(t, -2))(l)
    l = LSTM(128,return_sequences=True,activation='relu')(l)
    l = LSTM(128,return_sequences=True,activation='relu')(l)
    l = Flatten()(l)
    outputs = Dense(11,activation='softmax')(l)

    model = Model(inputs,outputs)
    model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])
    model.summary()
    return model

In [23]:
m1 = model1()

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 128, 2)            0         
_________________________________________________________________
batch_normalization_5 (Batch (None, 128, 2)            8         
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 124, 128)          1408      
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 41, 128)           0         
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 37, 128)           82048     
_________________________________________________________________
lstm_9 (LSTM)                (None, 37, 128)           131584    
_________________________________________________________________
lstm_10 (LSTM)               (None, 37, 128)           1315

In [ ]:
m1.fit(x=x_tr,y=y_tr,validation_data=[x_ts,y_ts],batch_size=128,epochs=10,shuffle='batch')

Train on 198000 samples, validate on 22000 samples
Epoch 1/10
 43136/198000 [=====>........................] - ETA: 9:25 - loss: 1.8334 - accuracy: 0.3034